In [1]:
import torch, numpy as np
from surrogate_helpers import VanillaModule, Dataset2DThermal
from surrogate_helpers import unpack_sym

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
ms_file = 'data/feature_engineering_data.h5'

# Load data
R_range_train = [1/100., 1/50., 1/20., 1/10., 1/5., 1/2., 2, 5, 10, 20, 50, 100]
train_data = Dataset2DThermal(ms_file, R_range_train, 'train_set', feature_idx=None)
R_range_val = np.concatenate([np.arange(2, 101, dtype=int), 1. / np.arange(2, 101, dtype=int)])
val_data = Dataset2DThermal(ms_file, R_range_val, 'benchmark_set', feature_idx=None)

train_x, train_y = train_data.features.to(device), train_data.kappa.to(device)
val_x, val_y = val_data.features.to(device), val_data.kappa.to(device)

In [ ]:
model_file = 'data/vrnn_therm2D.pt'

ann_model = torch.load(model_file, map_location=device).to(device=device, dtype=torch.float32)
model = VanillaModule(ann_model).to(device=device, dtype=torch.float32)
model.eval()

with torch.inference_mode():
    train_pred = model(train_x)
    val_pred = model(val_x)

In [ ]:
idx = np.random.randint(0, 297000)
print(idx)
print(f"val_y:            {val_y[idx, :].cpu().numpy()}")
print(f"val_pred:         {val_pred[idx, :].cpu().numpy()}")
print(f"val_y Eigenvalues: {torch.linalg.eigvalsh(unpack_sym(val_y[idx, :], dim=2)).cpu().numpy()}")
print(f"val_pred Eigenvalues: {torch.linalg.eigvalsh(unpack_sym(val_pred[idx, :], dim=2)).cpu().numpy()}")

In [ ]:
import sys
sys.path.append('./feature_engineering/')
import feature_extraction as extract
from utils import MicrostructureImageDataset

MS_train = MicrostructureImageDataset(file_path=ms_file, group_name= 'train_set', dset_name='image_data')
MS_bench = MicrostructureImageDataset(file_path=ms_file, group_name= 'benchmark_set', dset_name='image_data')

image_index = 1499
R = 73.658764

image = np.expand_dims(MS_bench[image_index].cpu().numpy(), axis=0)
train_feature = extract.full_computation(image) 
train_feature = torch.tensor(np.append(train_feature, [[1/R, R]], axis=1), device=device, dtype=torch.float32)

kappa_pred = unpack_sym(model(train_feature), dim=2)
print(kappa_pred.cpu().detach().numpy())


